Download the dataset from the link provided using !wget command

In [ ]:
!wget -q http://download.tensorflow.org/example_images/flower_photos.tgz

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
pathconf: Transport endpoint is not connected


In [34]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


*Unzip* the downloaded file using !unzip command

In [35]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [44]:
!ls

 Abbaqus
 Books
 Camscanner
'Coder Geeks'
'Colab Notebooks'
 data1
 data2
'Driving License'
'Educational Documents'
'Employment Documents'
 flower_photos
'Google Photos'
'Home Planning'
'Income Tax'
'Income tax investment points.gdoc'
'Interview Docs'
'IT Declaration 2021-2022.png'
'IT Junkies Projects 2019'
'Master'\''s Degree (Upgrad) Reference Docs'
'My docs'
'NodeJS Sample Project'
 Others
'PG ML'
'Recovery Keys'
'Rent Slips'
'SLR homes owner.jpg'
'upGrad ELEVATE Bootcamp resources, recodings'
'Upgrad Sessions'
'Zerodha Client Master Report.pdf'


In [60]:
!cwd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/bin/bash: cwd: command not found


In [ ]:
!tar -xvf flower_photos.tgz

Create a new directory 'data' to save your Train & Test data

In [9]:
import os
os.mkdir('data1')

In [10]:
import os
import shutil
import numpy as np
import glob   
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import  plot_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline


In [36]:
# Where all dataset is there
data_dir = 'flower_photos/'

# Training data dir
training_dir = 'data1/Train'

# Test data dir
testing_dir = 'data1/Test'

# Ratio of training and testing data
train_test_ratio = 0.8 


def split_dataset_into_test_and_train_sets(all_data_dir = data_dir, training_data_dir = training_dir, testing_data_dir=testing_dir, train_test_ratio = 0.8):
    # Recreate testing and training directories
    
    if not os.path.exists(training_data_dir):
            os.mkdir(training_data_dir)

    if not os.path.exists(testing_data_dir):
            os.mkdir(testing_data_dir)               
    
    num_training_files = 0
    num_testing_files = 0


    for subdir, dirs, files in os.walk(all_data_dir):
        
        category_name = os.path.basename(subdir)
        
        # print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
          continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name
        
        # creating subdir for each sub category
        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)   

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)
            
        file_list = glob.glob(os.path.join(subdir,'*.jpg'))

        #print(os.path.join(all_data_dir, subdir))
        print(str(category_name) + ' has ' + str(len(files)) + ' images') 
        random_set = np.random.permutation((file_list))
        # copy percentage of data from each category to train and test directory
        train_list = random_set[:round(len(random_set)*(train_test_ratio))] 
        test_list = random_set[-round(len(random_set)*(1-train_test_ratio)):]

  

        for lists in train_list : 
            shutil.copy(lists, training_data_dir + '/' + category_name + '/' )
            num_training_files += 1
  
        for lists in test_list : 
            shutil.copy(lists, testing_data_dir + '/' + category_name + '/' )
            num_testing_files += 1
  

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [13]:
split_dataset_into_test_and_train_sets()

roses has 641 images
sunflowers has 699 images
tulips has 799 images
daisy has 633 images
dandelion has 898 images
Processed 2935 training files.
Processed 735 testing files.


In [14]:
# Number of classes in dataset
num_classes = 5

def get_model():
    # Get base model 
    # Here we are using ResNet50 as base model | Actual softmax layer we do not have to include.
    base_model = ResNet50(weights='imagenet', include_top=False)
    
    # As we are using ResNet model only for feature extraction and not adjusting the weights
    # we freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = False
        
    # Get base model output 
    base_model_ouput = base_model.output
    
    # Adding our own layer 
    x = GlobalAveragePooling2D()(base_model_ouput)
    # Adding fully connected layer
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [16]:
# Get the model
model = get_model()
# Compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Summary of model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [45]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64

train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

Found 2935 images belonging to 5 classes.
Found 735 images belonging to 5 classes.


In [18]:
# Training the fully conncected layer for initial epochs
epochs = 5

# Training the model

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs)

Epoch 1/5
45/45 [==============================] - 73s 896ms/step - loss: 0.7739 - accuracy: 0.7229 - val_loss: 0.4318 - val_accuracy: 0.8565
Epoch 2/5
45/45 [==============================] - 40s 879ms/step - loss: 0.4159 - accuracy: 0.8558 - val_loss: 0.3762 - val_accuracy: 0.8665
Epoch 3/5
45/45 [==============================] - 39s 875ms/step - loss: 0.3430 - accuracy: 0.8899 - val_loss: 0.3161 - val_accuracy: 0.8991
Epoch 4/5
45/45 [==============================] - 41s 906ms/step - loss: 0.2952 - accuracy: 0.9000 - val_loss: 0.2869 - val_accuracy: 0.9006
Epoch 5/5
45/45 [==============================] - 39s 877ms/step - loss: 0.2727 - accuracy: 0.9098 - val_loss: 0.2721 - val_accuracy: 0.9091


In [19]:
# More fine tuning the model
# Training the model after 140 layers
# Generally ResNet is good at extracting lower level features so we are not fine tuning initial layers
epochs = 10

split_at = 140
for layer in model.layers[:split_at]: 
  layer.trainable = False
for layer in model.layers[split_at:]: 
  layer.trainable = True
    
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate, if we are fine tuning the initial layers
sgd = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
45/45 [==============================] - 46s 905ms/step - loss: 0.3438 - accuracy: 0.8962 - val_loss: 0.2718 - val_accuracy: 0.9162
Epoch 2/10
45/45 [==============================] - 40s 895ms/step - loss: 0.2309 - accuracy: 0.9352 - val_loss: 0.2601 - val_accuracy: 0.9148
Epoch 3/10
45/45 [==============================] - 40s 891ms/step - loss: 0.1687 - accuracy: 0.9502 - val_loss: 0.2406 - val_accuracy: 0.9176
Epoch 4/10
45/45 [==============================] - 40s 891ms/step - loss: 0.1263 - accuracy: 0.9669 - val_loss: 0.2400 - val_accuracy: 0.9261
Epoch 5/10
45/45 [==============================] - 40s 895ms/step - loss: 0.1007 - accuracy: 0.9735 - val_loss: 0.2243 - val_accuracy: 0.9332
Epoch 6/10
45/45 [==============================] - 40s 885ms/step - loss: 0.0872 - accuracy: 0.9753 - val_loss: 0.2197 - val_accuracy: 0.9318
Epoch 7/10
45/45 [==============================] - 40s 888ms/step - loss: 0.0702 - accuracy: 0.9868 - val_loss: 0.2161 - val_accuracy: 0.9389

In [ ]:
print('Training complete')

Training complete


In [23]:
os.mkdir('data1')

In [25]:
!pwd

/content/drive/My Drive


In [26]:
model.save('data1/')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: data1/assets
